In [2]:
import datetime
import pandas as pd
import numpy as np
import random as rand
from bayes_opt import BayesianOptimization #pip install bayesian-optimization

#테스트 파일 로딩
coin_name = "KRW-XRP"
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
print(df_org.shape)
print(df_org.columns)

(199670, 32)
Index(['Unnamed: 0', 't', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99',
       'wma7', 'wma25', 'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s',
       'macd_d', 'rsi7', 'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh',
       'bhi', 'bl', 'bli', 'bm', 'bw', 'vwap'],
      dtype='object')


In [3]:
#타임스켐프 시간 변환
def get_time_hhmmss(mili_time):
    mili_time = float(mili_time)/1000
    KST = datetime.timezone(datetime.timedelta(hours=9))
    dt = datetime.datetime.fromtimestamp(mili_time, tz=KST)
    timeline = str(dt.strftime('%D %H:%M:%S'))
    return timeline

#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)

#수익률 보정: 총매수금액 200 coin(xrp) 기준으로 조정
def adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate):
    open_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_amt = open_amt_list[len(open_amt_list)-1]
    adj_revenue = (200 * revenue) / max_amt
    return adj_revenue

def run_test(config):
    revenue_rate  = config['revenue_rate']
    max_loss_rate = config['max_loss_rate']
    increace_rate = config['increace_rate']
    buy_cnt_limit = int(config['buy_cnt_limit'])
    buy_amt_unit  = config['buy_amt_unit']
    trade_fee = 0.001 #거래수수료
    close = 400
    buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)

    buy_cnt = 0 
    buy_price = 0
    buy_amt = 0
    revenue = 0
    revenue_t = 0

#     df = df_org.iloc[df_org.shape[0]-28800:]
    df = df_org.iloc[df_org.shape[0]-144000:df_org.shape[0]-124000,]
    # df = df_org2
    for i in range(0, df.shape[0]-1):
        close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
        close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4) #다음 분의 종가로 산다
        wma7 = round(df.iloc[i:i+1,]['wma7'].values[0],4)
        wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4)
        vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
        macd_s = round(df.iloc[i:i+1,]['macd_s'].values[0],4)
        macd_d = round(df.iloc[i:i+1,]['macd_d'].values[0],4)

        #stop loss
        loss = buy_price - close2*buy_amt
        if loss > max_loss:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #take profit
        tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
        if buy_cnt > 0 and tp_revenue > 0:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #buy
        if buy_cnt < buy_cnt_limit and close2 < vwap and close2 < wma7 and wma7 > wma99:
            temp_amt = buy_amt_unit + buy_amt*increace_rate
            buy_price = round(buy_price + close2*temp_amt,4)
            buy_amt = round(buy_amt + temp_amt,4)
            buy_cnt = buy_cnt + 1 
        
    return adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate)

In [4]:
def black_box_function(revenue_rate, max_loss_rate, increace_rate, buy_cnt_limit, buy_amt_unit):
    config_data = {
        'revenue_rate':revenue_rate,
        'max_loss_rate':max_loss_rate,
        'increace_rate':increace_rate,
        'buy_cnt_limit':buy_cnt_limit,
        'buy_amt_unit':buy_amt_unit
    }
    revenue = run_test(config_data)
    return revenue

pbounds = {
    'revenue_rate':(0.005,0.015),
    'max_loss_rate':(0.05,0.3),
    'increace_rate':(0.1,0.4),
    'buy_cnt_limit':(5,20),
    'buy_amt_unit':(15,40)
    
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1
)

#실행
optimizer.maximize(
    init_points=5,
    n_iter=50
)

|   iter    |  target   | buy_am... | buy_cn... | increa... | max_lo... | revenu... |
-------------------------------------------------------------------------------------
| 1         | 6.765e+03 | 25.43     | 15.8      | 0.1       | 0.1256    | 0.006468  |
| 2         | 6.194e+03 | 17.31     | 7.794     | 0.2037    | 0.1492    | 0.01039   |
| 3         | 6.524e+03 | 25.48     | 15.28     | 0.1613    | 0.2695    | 0.005274  |
| 4         | 5.815e+03 | 31.76     | 11.26     | 0.2676    | 0.0851    | 0.006981  |
| 5         | 5.581e+03 | 35.02     | 19.52     | 0.194     | 0.2231    | 0.01376   |
| 6         | 6.017e+03 | 24.84     | 16.99     | 0.3683    | 0.1274    | 0.01117   |
| 7         | 867.5     | 26.14     | 15.95     | 0.1       | 0.05      | 0.005     |
| 8         | 6.906e+03 | 25.48     | 15.49     | 0.2937    | 0.09837   | 0.00643   |
| 9         | 6.561e+03 | 24.71     | 18.89     | 0.3546    | 0.1966    | 0.007194  |
| 10        | 5.042e+03 | 25.34     | 15.74     | 0.10

In [5]:
target_list = []
i=0
for res in optimizer.res:
    target_list.append([res["target"], i])
    i=i+1
target_list.sort(reverse=True)    
target_list

[[9281.123679070473, 24],
 [7622.135385680692, 12],
 [7608.219518710377, 46],
 [7520.748086898137, 50],
 [7424.666707087008, 23],
 [7173.437680386696, 43],
 [7170.432046865212, 39],
 [7070.654084246102, 40],
 [6905.717848844099, 7],
 [6871.178629550159, 28],
 [6851.3850832464095, 52],
 [6832.856043457935, 18],
 [6820.237075112052, 47],
 [6814.466471936725, 51],
 [6814.231817181541, 14],
 [6765.143519687881, 0],
 [6747.2019985708675, 16],
 [6713.076217472259, 49],
 [6643.617914563552, 41],
 [6606.983922910553, 21],
 [6560.992700417096, 8],
 [6540.112903768664, 29],
 [6523.663846358513, 2],
 [6496.011028206244, 31],
 [6493.417689161555, 25],
 [6325.276180894296, 22],
 [6291.990814822915, 54],
 [6284.259044826936, 27],
 [6193.746246199761, 1],
 [6048.785105518955, 32],
 [6016.546726576784, 5],
 [5916.134443602351, 15],
 [5821.869011618208, 53],
 [5814.7968122963775, 3],
 [5786.8942833608735, 35],
 [5711.194424009611, 33],
 [5692.003962753999, 20],
 [5647.557989364019, 37],
 [5636.63307878

In [7]:
optimizer.res[24]

{'target': 9281.123679070473,
 'params': {'buy_amt_unit': 28.91339164826654,
  'buy_cnt_limit': 18.490269187688337,
  'increace_rate': 0.10318682066581343,
  'max_loss_rate': 0.2862038139314399,
  'revenue_rate': 0.008092014657265029}}